<h1>Trabajo Grupal - Desarrollo de Aplicaciones con Visión Artificial</h1>
<span>Miembros del grupo: <strong>Mateo Ignacio Ismodes Sole, Jimena Tagle</strong></span>

---
<h2>Importación de dependencias y dataset

In [1]:
# Importacion de modulos
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests
import numpy as np
import random
import shutil
import pandas as pd

In [2]:
# Directorios de Data para Modelo
source_folder_1 = '/content/drive/MyDrive/Trees_Dataset/train/' # Data de entrenamiento
source_folder_2 = '/content/drive/MyDrive/Trees_Dataset/valid/' # Data de validación
source_folder_3 = '/content/drive/MyDrive/Trees_Dataset/test/'  # Data de pruebas

# Directorio destino en Google Colab
destination_folder = '/content/'

# Data de directorios es copiada a directorio de Google Colab
shutil.copytree(source_folder_1, os.path.join(destination_folder, 'train'))
shutil.copytree(source_folder_2, os.path.join(destination_folder, 'valid'))
shutil.copytree(source_folder_3, os.path.join(destination_folder, 'test'))

'/content/test'

---
<h2>Importación del modelo preeentrenado YOLO v5 large y de sus dependencias</h2>

In [3]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 16003, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 16003 (delta 21), reused 20 (delta 13), pack-reused 15967
Receiving objects: 100% (16003/16003), 14.66 MiB | 26.02 MiB/s, done.
Resolving deltas: 100% (10987/10987), done.


In [4]:
%cd yolov5/
!pwd
!pip install -r requirements.txt

/content/yolov5
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.9/618.9 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00


In [5]:
# YOLO v5 requiere un file .yaml con información de los paths y las clases
%%writefile ../data.yaml
train: ../train/images
val: ../valid/images
nc: 1
names: ['Tree'] # nombres de las clases

Writing ../data.yaml


---
<h2>Definición de funciones necesarias</h2>

In [6]:
def set_res_dir():
    # Directory to store results
    res_dir_count = len(glob.glob('runs/train/*'))
    print(f"Current number of result directories: {res_dir_count}")
    if TRAIN:
        RES_DIR = f"results_{res_dir_count+1}"
        print(RES_DIR)
    else:
        RES_DIR = f"results_{res_dir_count}"
    return RES_DIR

In [7]:
def show_inf_dir_name():
    # Directory to store inference results.
    infer_dir_count = len(glob.glob('runs/detect/*'))
    INFER_DIR = f"inference_{infer_dir_count+1}"
    return INFER_DIR

In [8]:
def csv_to_df(inf_path, inf_dir):
  df = pd.read_csv(os.path.join(inf_path, inf_dir, 'predictions.csv'), header = None)
  return df

---
<h2>Finetuning del Modelo preentrenado</h2>

In [9]:
%cd yolov5/
!pwd

[Errno 2] No such file or directory: 'yolov5/'
/content/yolov5
/content/yolov5


In [10]:
TRAIN = True # Se especifica la intención para el modelo
EPOCHS = 100 # Se especifica el número de épocas de entrenamiento

In [11]:
RES_DIR = set_res_dir()
if TRAIN:
    !python train.py --data ../data.yaml --weights yolov5x.pt \
    --img 640 --epochs {EPOCHS} --batch-size 16 --name {RES_DIR}

Current number of result directories: 0
results_1
train: weights=yolov5x.pt, cfg=, data=../data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=results_1, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-226-gdd9e338 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, ancho

---
<h2>Detección con YOLO v5 large con fine-tuning</h2>

In [12]:
INF_DIR = show_inf_dir_name()

In [13]:
!python detect.py --source ../test/images/ --weights runs/train/results_1/weights/best.pt --conf 0.25 --name {INF_DIR} --save-csv

detect: weights=['runs/train/results_1/weights/best.pt'], source=../test/images/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=inference_1, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-226-gdd9e338 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
image 1/10 /content/test/images/8065-0.jpg: 640x480 1 Tree, 70.5ms
image 2/10 /content/test/images/8089-0.jpg: 640x480 1 Tree, 44.2ms
image 3/10 /content/test/images/8122-0.jpg: 640x480 1 Tree, 44.1ms
image 4/10 /content/test/images/8223-0.jpg: 640x480 3 Trees, 44.1ms
image 5/10 /content/test/images/8227-0.jpg: 640x48

In [14]:
# Extraer csv y convertirlo a dataframe de Pandas
INF_PATH = "runs/detect"
df = csv_to_df(INF_PATH, INF_DIR)
df
print("El número de árboles detectados es:",len(df))

El número de árboles detectados es: 14


<h2>Cargando Modelo xl de 100 epochs (Para cargar pesos previamente obtenidos.)</h2>

In [ ]:
source_folder_4 = '/content/drive/MyDrive/Model/train/'
destination_folder = '/content/yolov5/runs/'

shutil.copytree(source_folder_4, os.path.join(destination_folder, 'train'))


'/content/yolov5/runs/train'

In [ ]:
INF_DIR = show_inf_dir_name()
!python detect.py --source ../test_1/ --weights runs/train/results_xl_100/weights/best.pt --conf 0.15 --name {INF_DIR} --save-csv

detect: weights=['runs/train/results_xl_100/weights/best.pt'], source=../test_1/, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.15, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=inference_4, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-226-gdd9e338 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 322 layers, 86173414 parameters, 0 gradients, 203.8 GFLOPs
image 1/8 /content/test_1/200907-0.jpg: 640x480 3 Trees, 86.9ms
image 2/8 /content/test_1/212167-0.jpg: 640x480 (no detections), 45.8ms
image 3/8 /content/test_1/212245-0.jpg: 640x480 (no detections), 45.8ms
image 4/8 /content/test_1/212465-0.jpg: 640x480 1 Tree, 43.3ms
image 5/8 /content/test_1/216565-0.jpg: 640x480 